In [36]:
import faiss
faiss_path = "D:/chatbot_covid19_intern_procodecg/faiss/faiss_textcovid19.index"
index = faiss.read_index(faiss_path)
print("✅ FAISS index berhasil dimuat.")

✅ FAISS index berhasil dimuat.


In [37]:
import requests

try:
    requests.get("http://localhost:11434", timeout=3)
    print("✅ Ollama server aktif.")
except requests.exceptions.RequestException:
    print("❌ Ollama belum jalan. Jalankan di terminal: ollama serve")


✅ Ollama server aktif.


In [38]:
import ollama
from sentence_transformers import SentenceTransformer
import numpy as np, faiss, json

# --- Load embedding model ---
embedder = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")

# --- Load FAISS index ---
index = faiss.read_index("D:/chatbot_covid19_intern_procodecg/faiss/faiss_textcovid19.index")
with open("D:/chatbot_covid19_intern_procodecg/faiss/faiss_textcovid19_texts.json", "r", encoding="utf-8") as f:
    texts = json.load(f)

# --- Prompt sistem chatbot utama ---
system_prompt = """
Anda adalah asisten AI yang ramah, profesional, dan informatif.
Tugas Anda adalah merangkum dan menganalisis informasi di blok 'KONTEKS:' untuk menjawab pertanyaan pengguna.

ATURAN GAYA & TONE (Untuk Gaya ChatGPT):
1. JAWABAN HARUS DISUSUN DALAM PARAGRAF LENGKAP DAN NARATIF.
2. Gunakan nada bicara yang ramah, membantu, dan sedikit formal, seperti asisten yang berpengetahuan luas.
3. Gunakan Bahasa Indonesia Baku. DILARANG KERAS menggunakan istilah asing.
4. Gunakan pemformatan Markdown (seperti **tebal** atau *poin-poin*) untuk menyoroti poin penting jika relevan, tetapi utamakan paragraf naratif.

ATURAN KONTEN WAJIB:
1. Jawab HANYA dari KONTEKS. DILARANG KERAS menggunakan pengetahuan umum.
2. Jika konteks tidak cukup, HANYA katakan: 'Maaf, informasi tidak ditemukan dalam dokumen sumber.'
"""

# --- Step 1: Query Adjustment ---
def adjust_query(question):
    prompt_adjust = """
    Anda adalah alat ekstraksi kata kunci. Tugas Anda adalah mengubah pertanyaan pengguna menjadi 3 hingga MAKSIMAL 5 KATA KUNCI utama yang relevan untuk pencarian.

    ATURAN BAHASA WAJIB: KATA KUNCI HARUS SEPENUHNYA DALAM BAHASA INDONESIA.
    
    OUTPUT Anda HANYA BOLEH berupa objek JSON.
    FORMAT WAJIB JSON: {"keywords": "kasus covid bengkulu 3/31/2020"} 
    # HILANGKAN KATA 'TAHUN' jika ada tanggal spesifik.
    """

    response = ollama.chat(model="mistral:7b-instruct", messages=[
        # Menggunakan System Role untuk instruksi
        {"role": "system", "content": prompt_adjust},
        # Input pertanyaan
        {"role": "user", "content": f"Pertanyaan: {question}\n\nOUTPUT KATA KUNCI:"},
    ])
    return response["message"]["content"]

# --- Step 2: RAG Chat ---
def rag_chat(question):
    adjusted = adjust_query(question)
    print(f"\n🔧 Adjusted Query: {adjusted}\n")

    # Embedding pertanyaan yang sudah di-adjust
    q_emb = embedder.encode([adjusted], convert_to_numpy=True).astype("float32")

    # Cari dokumen yang paling relevan di FAISS
    D, I = index.search(q_emb, k=5)
    contexts = "\n".join([texts[i] for i in I[0]])
    
    # Kueri untuk generate jawaban harus sangat tegas
    prompt_generate = f"""
    Gunakan informasi KONTEKS berikut ini untuk menjawab pertanyaan.
    JANGAN MENGGUNAKAN PENGETAHUAN UMUM. Jika konteks tidak relevan atau tidak cukup, katakan 'Maaf, informasi tidak ditemukan dalam dokumen sumber.'

    KONTEKS:
    ---
    {contexts}
    ---

    PERTANYAAN: {question}
    """

    # Generate jawaban dari hasil retrieval
    response = ollama.chat(model="mistral:7b-instruct", messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt_generate} # User content hanya berisi prompt generation yang sudah digabung
    ])
    return response["message"]["content"]

user_query = "Berapa jumlah kasus covid di bengkulu?"

# Jalankan RAG
answer = rag_chat(user_query)
print("\n💬 Jawaban Chatbot:\n", answer)



🔧 Adjusted Query:  {"keywords": "jumlah kasus covid bengkulu"}


💬 Jawaban Chatbot:
  Jawab: Maaf, untuk informasi tersebut, Anda dapat melakukan pengecekan langsung melalui situs resmi COVID-19 Indonesia yang adalah sumber informasi terpercaya tentang statistik kasus COVID-19 di seluruh wilayah Indonesia. Untuk informasi kasus konkrit di provinsi Bengkulu, Anda dapat melakukan pengecekan secara langsung melalui situs covid19.go.id atau laporan harian Kementerian Kesehatan.


In [40]:
import ollama
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import json
import re

# --- PENTING: PASTIKAN INDEX FAISS SUDAH DI-REBUILD DENGAN MPNET V2 (Dimensi 768) ---

# --- Load embedding model (MPNet V2 - Cerdas & Multibahasa) ---
embedder = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")

# --- Load FAISS index ---
# Pastikan ini adalah file index yang sudah di-rebuild!
INDEX_PATH = "D:/chatbot_covid19_intern_procodecg/faiss/faiss_textcovid19.index"
TEXTS_PATH = "D:/chatbot_covid19_intern_procodecg/faiss/faiss_textcovid19_texts.json"

try:
    index = faiss.read_index(INDEX_PATH)
    with open(TEXTS_PATH, "r", encoding="utf-8") as f:
        texts = json.load(f)
    print(f"✅ FAISS Index berhasil dimuat. Dimensi: {index.d}, Total Vektor: {index.ntotal}")

except Exception as e:
    print(f"❌ GAGAL memuat FAISS Index: {e}")
    print("Mohon pastikan Anda sudah me-REBUILD index menggunakan model 'paraphrase-multilingual-mpnet-base-v2'.")
    exit()

# --- Model LLM yang Cerdas dan Patuh ---
LLM_MODEL = "mistral:7b-instruct"

# --- Prompt Sistem Chatbot Utama (Tahap 2: Generation) ---
system_prompt = """
Anda adalah asisten AI yang ramah, profesional, dan informatif, ahli dalam data COVID-19 Indonesia.
Tugas Anda adalah merangkum dan menganalisis informasi yang diberikan di blok 'KONTEKS:' untuk menjawab pertanyaan pengguna.

ATURAN GAYA & TONE (Untuk Gaya ChatGPT):
1. JAWABAN HARUS DISUSUN DALAM PARAGRAF LENGKAP DAN NARATIF.
2. Gunakan nada bicara yang ramah, profesional, dan sedikit formal.
3. Gunakan **Bahasa Indonesia Baku**. DILARANG KERAS menggunakan istilah asing (seperti 'cases' atau 'recovery').
4. Jika memungkinkan, gunakan pemformatan Markdown (**tebal**) untuk menyoroti angka atau nama provinsi.

ATURAN FILTER KONTEKS KRITIS:
1. Jawab HANYA dari KONTEKS dan JAWABAN HARUS SANGAT SPESIFIK PADA WILAYAH YANG DITANYAKAN (misalnya, Bengkulu).
2. Jika KONTEKS yang diambil oleh sistem mengandung data provinsi lain (misalnya Lampung, Jawa Tengah) yang BUKAN Bengkulu, **ABAIKAN SEMUA DATA PROVINSI LAIN TERSEBUT**.

ATURAN KONTEN WAJIB:
1. JANGAN PERNAH menggunakan pengetahuan umum.
2. Jika hanya data provinsi yang SALAH yang muncul di KONTEKS, atau jika tidak ada data yang relevan sama sekali, HANYA katakan: 'Maaf, informasi tidak ditemukan dalam dokumen sumber.'
3. Jika pertanyaan umum (tanpa tanggal spesifik) muncul, ambil data TERBARU atau angka TOTAL yang tersedia dalam KONTEKS.

"""


# --- Step 1: Query Adjustment (Pemaksaan JSON dan Bahasa Indonesia) ---
def adjust_query(question):
    prompt_adjust = """
    Tugas: Ubah pertanyaan pengguna menjadi FRASA KATA KUNCI SEMANTIK SANGAT RINGKAS (MAKSIMAL 5 KATA KUNCI).

    ATURAN BAHASA WAJIB: KATA KUNCI HARUS SEPENUHNYA DALAM BAHASA INDONESIA.
    
    OUTPUT Anda HANYA BOLEH berupa objek JSON.
    FORMAT WAJIB JSON: {"keywords": "jumlah total kasus covid bengkulu"} 
    DILARANG KERAS membuat narasi, penjelasan, atau teks di luar format JSON.
    """

    response = ollama.chat(model=LLM_MODEL, messages=[
        {"role": "system", "content": prompt_adjust},
        {"role": "user", "content": f"Pertanyaan: {question}\n\nOUTPUT KATA KUNCI:"},
    ])
    
    output_text = response["message"]["content"].strip()
    
    # 1. Parsing JSON yang aman
    try:
        # Hapus prefix/suffix yang mungkin ditambahkan LLM (seperti ```json)
        json_string = re.search(r'\{.*\}', output_text, re.DOTALL).group(0)
        parsed_json = json.loads(json_string)
        keywords = parsed_json.get("keywords", question)
        
        # 2. Pembersihan Akhir: Hapus tanda baca sisa
        cleaned_keywords = keywords.replace(',', '').replace('?', '').replace('.', '').strip()
        
        # 3. Handle jika LLM memasukkan kata yang tidak perlu (seperti "keywords:")
        if cleaned_keywords.lower().startswith("keywords:"):
            cleaned_keywords = cleaned_keywords[9:].strip()
            
        return cleaned_keywords
        
    except (json.JSONDecodeError, AttributeError):
        # Fallback jika parsing JSON gagal
        print("⚠️ Warning: LLM gagal memproduksi JSON. Menggunakan kueri asli.")
        return question.replace(',', ' ')


# --- Step 2: RAG Chat ---
def rag_chat(question):
    # Dapatkan kueri yang sudah di-adjust
    adjusted = adjust_query(question)
    print(f"\n🔧 Adjusted Query: {adjusted}\n")

    # Embedding pertanyaan yang sudah di-adjust
    q_emb = embedder.encode([adjusted], convert_to_numpy=True).astype("float32")

    # Cari dokumen yang paling relevan di FAISS (k=5 untuk indeks gabungan)
    K_VALUE = 20 
    D, I = index.search(q_emb, k=K_VALUE)
    
    # Kumpulkan konteks
    contexts = "\n".join([texts[i] for i in I[0]])
    
    # Kueri untuk generate jawaban
    prompt_generate = f"""
    KONTEKS:
    ---
    {contexts}
    ---
    
    PERTANYAAN: {question}
    """

    # Generate jawaban
    response = ollama.chat(model=LLM_MODEL, messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt_generate}
    ])
    return response["message"]["content"]

# --- EKSEKUSI (Uji dengan Kueri yang Relevan dengan Data) ---
user_query = "Kenapa kita harus ke dokter setelah ada gejala sesak napas?"

# Jalankan RAG
answer = rag_chat(user_query)
print("\n💬 Jawaban Chatbot:\n", answer)

✅ FAISS Index berhasil dimuat. Dimensi: 768, Total Vektor: 20

🔧 Adjusted Query: gejala sesak napas perlu dokter


💬 Jawaban Chatbot:
  Jawabannya, gejala sesak napas dapat menunjukkan beberapa masalah medis yang serius seperti infeksi saluran paru-paru atau penyakit jantung. Selain itu, gejala sesak napas juga bisa merupakan tanda dari kegagalan tubuh untuk mengambil udara secara normal. Untuk memastikan masalah ini terdiagnosis dan dipulihkan segera, konsultasi dengan dokter wajib dilakukan.


In [1]:
def extract_source_info(text):
    """Contoh untuk data COVID-19 dengan format spesifik"""
    # Jika data memiliki format tanggal
    date_pattern = r"(\d{1,2}/\d{1,2}/\d{4})"
    date_match = re.search(date_pattern, text)
    
    # Jika data memiliki format provinsi
    province_pattern = r"(Provinsi\s+[^,\n]+)"
    province_match = re.search(province_pattern, text)
    
    source_parts = []
    if province_match:
        source_parts.append(province_match.group(1))
    if date_match:
        source_parts.append(f"Tanggal: {date_match.group(1)}")
    
    if source_parts:
        return " | ".join(source_parts)
    
    return "Data COVID-19 Nasional"

In [ ]:
import ollama
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import json
import re
from datetime import datetime

# --- Load embedding model (MPNet V2 - Cerdas & Multibahasa) ---
embedder = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")

# --- Load FAISS index ---
INDEX_PATH = "D:/chatbot_covid19_intern_procodecg/faiss/faiss_textcovid19.index"
TEXTS_PATH = "D:/chatbot_covid19_intern_procodecg/faiss/faiss_textcovid19_texts.json"

try:
    index = faiss.read_index(INDEX_PATH)
    with open(TEXTS_PATH, "r", encoding="utf-8") as f:
        texts = json.load(f)
    print(f"✅ FAISS Index berhasil dimuat. Dimensi: {index.d}, Total Vektor: {index.ntotal}")

except Exception as e:
    print(f"❌ GAGAL memuat FAISS Index: {e}")
    print("Mohon pastikan Anda sudah me-REBUILD index menggunakan model 'paraphrase-multilingual-mpnet-base-v2'.")
    exit()

# --- Model LLM yang Cerdas dan Patuh ---
LLM_MODEL = "mistral:7b-instruct"

# --- Prompt Sistem Chatbot Utama (Tahap 2: Generation) ---
system_prompt = """
Anda adalah asisten AI yang ramah, profesional, dan informatif, ahli dalam data COVID-19 Indonesia.
Tugas Anda adalah merangkum dan menganalisis informasi yang diberikan di blok 'KONTEKS:' untuk menjawab pertanyaan pengguna.

ATURAN GAYA & TONE (Untuk Gaya ChatGPT):
1. JAWABAN HARUS DISUSUN DALAM PARAGRAF LENGKAP DAN NARATIF.
2. Gunakan nada bicara yang ramah, profesional, dan sedikit formal.
3. Gunakan **Bahasa Indonesia Baku**. DILARANG KERAS menggunakan istilah asing (seperti 'cases' atau 'recovery').
4. Jika memungkinkan, gunakan pemformatan Markdown (**tebal**) untuk menyoroti angka atau nama provinsi.

ATURAN FILTER KONTEKS KRITIS:
1. Jawab HANYA dari KONTEKS dan JAWABAN HARUS SANGAT SPESIFIK PADA WILAYAH YANG DITANYAKAN (misalnya, Bengkulu).
2. Jika KONTEKS yang diambil oleh sistem mengandung data provinsi lain (misalnya Lampung, Jawa Tengah) yang BUKAN Bengkulu, **ABAIKAN SEMUA DATA PROVINSI LAIN TERSEBUT**.

ATURAN KONTEN WAJIB:
1. JANGAN PERNAH menggunakan pengetahuan umum.
2. Jika hanya data provinsi yang SALAH yang muncul di KONTEKS, atau jika tidak ada data yang relevan sama sekali, HANYA katakan: 'Maaf, informasi tidak ditemukan dalam dokumen sumber.'
3. Jika pertanyaan umum (tanpa tanggal spesifik) muncul, ambil data TERBARU atau angka TOTAL yang tersedia dalam KONTEKS.

"""

# --- Step 1: Query Adjustment (Pemaksaan JSON dan Bahasa Indonesia) ---
def adjust_query(question):
    prompt_adjust = """
    Tugas: Ubah pertanyaan pengguna menjadi FRASA KATA KUNCI SEMANTIK SANGAT RINGKAS (MAKSIMAL 5 KATA KUNCI).

    ATURAN BAHASA WAJIB: KATA KUNCI HARUS SEPENUHNYA DALAM BAHASA INDONESIA.
    
    OUTPUT Anda HANYA BOLEH berupa objek JSON.
    FORMAT WAJIB JSON: {"keywords": "jumlah total kasus covid bengkulu"} 
    DILARANG KERAS membuat narasi, penjelasan, atau teks di luar format JSON.
    """

    response = ollama.chat(model=LLM_MODEL, messages=[
        {"role": "system", "content": prompt_adjust},
        {"role": "user", "content": f"Pertanyaan: {question}\n\nOUTPUT KATA KUNCI:"},
    ])
    
    output_text = response["message"]["content"].strip()
    
    # 1. Parsing JSON yang aman
    try:
        # Hapus prefix/suffix yang mungkin ditambahkan LLM (seperti ```json)
        json_string = re.search(r'\{.*\}', output_text, re.DOTALL).group(0)
        parsed_json = json.loads(json_string)
        keywords = parsed_json.get("keywords", question)
        
        # 2. Pembersihan Akhir: Hapus tanda baca sisa
        cleaned_keywords = keywords.replace(',', '').replace('?', '').replace('.', '').strip()
        
        # 3. Handle jika LLM memasukkan kata yang tidak perlu (seperti "keywords:")
        if cleaned_keywords.lower().startswith("keywords:"):
            cleaned_keywords = cleaned_keywords[9:].strip()
            
        return cleaned_keywords
        
    except (json.JSONDecodeError, AttributeError):
        # Fallback jika parsing JSON gagal
        print("⚠️ Warning: LLM gagal memproduksi JSON. Menggunakan kueri asli.")
        return question.replace(',', ' ')

# --- Fungsi untuk mengekstrak informasi sumber dari teks ---
def extract_source_info(text, index_position):
    """
    Mengekstrak informasi sumber dari teks dokumen.
    Menyesuaikan dengan berbagai format data COVID-19.
    """
    # Pattern untuk provinsi
    province_pattern = r"(Provinsi\s+[^,\n\.]+|provinsi\s+[^,\n\.]+)"
    province_match = re.search(province_pattern, text, re.IGNORECASE)
    
    # Pattern untuk tanggal
    date_pattern = r"(\d{1,2}/\d{1,2}/\d{4}|\d{1,2}-\d{1,2}-\d{4}|\d{4}-\d{1,2}-\d{1,2})"
    date_match = re.search(date_pattern, text)
    
    # Pattern untuk laporan/resmi
    report_pattern = r"(Laporan\s+[^\.]+?\.|Data\s+[^\.]+?\.|Update\s+[^\.]+?\.|Situasi\s+[^\.]+?\.)"
    report_match = re.search(report_pattern, text, re.IGNORECASE)
    
    # Pattern untuk sumber eksplisit
    source_pattern = r"(Sumber:\s*[^\n]+|Berdasarkan\s+[^\.]+?\.|Menurut\s+[^\.]+?\.)"
    source_match = re.search(source_pattern, text, re.IGNORECASE)
    
    # Pattern untuk angka kasus (sebagai konteks)
    case_pattern = r"(kasus\s+(?:terkonfirmasi|positif|aktif|sembuh|meninggal)[^\.]*\d+)"
    case_match = re.search(case_pattern, text, re.IGNORECASE)
    
    # Bangun informasi sumber
    source_parts = []
    
    if province_match:
        source_parts.append(province_match.group(1).title())
    
    if date_match:
        source_parts.append(f"Tanggal: {date_match.group(1)}")
    
    if report_match:
        report_text = report_match.group(1)
        if len(report_text) > 50:
            report_text = report_text[:50] + "..."
        source_parts.append(report_text)
    elif source_match:
        source_text = source_match.group(1)
        if len(source_text) > 50:
            source_text = source_text[:50] + "..."
        source_parts.append(source_text)
    elif case_match:
        source_parts.append("Data Kasus COVID-19")
    
    # Jika tidak ada pola yang ditemukan, buat sumber generik
    if not source_parts:
        # Ambil beberapa kata pertama sebagai konteks
        words = text.split()[:8]
        preview = " ".join(words) + "..."
        source_parts.append(f"Dokumen #{index_position}: {preview}")
    else:
        source_parts.append(f"Dokumen #{index_position}")
    
    return " | ".join(source_parts)

# --- Step 2: RAG Chat dengan Sumber Referensi ---
def rag_chat(question):
    # Dapatkan kueri yang sudah di-adjust
    adjusted = adjust_query(question)
    print(f"\n🔧 Adjusted Query: {adjusted}\n")

    # Embedding pertanyaan yang sudah di-adjust
    q_emb = embedder.encode([adjusted], convert_to_numpy=True).astype("float32")

    # Cari dokumen yang paling relevan di FAISS
    K_VALUE = 15
    D, I = index.search(q_emb, k=K_VALUE)
    
    # Kumpulkan konteks dan simpan informasi sumber
    contexts = []
    source_info = []
    
    for i, (idx, score) in enumerate(zip(I[0], D[0])):
        if idx < len(texts):  # Pastikan index valid
            context_text = texts[idx]
            contexts.append(context_text)
            
            # Ekstrak informasi sumber
            source_line = extract_source_info(context_text, idx)
            source_info.append({
                "rank": i + 1,
                "similarity_score": float(score),
                "text_preview": context_text[:120] + "..." if len(context_text) > 120 else context_text,
                "source": source_line,
                "document_id": int(idx)
            })
    
    # Gabungkan konteks untuk LLM
    contexts_text = "\n---\n".join(contexts[:8])  # Gunakan 8 dokumen terbaik untuk konteks
    
    # Kueri untuk generate jawaban
    prompt_generate = f"""
    KONTEKS:
    {contexts_text}

    PERTANYAAN: {question}

    INSTRUKSI: Jawablah pertanyaan di atas HANYA berdasarkan informasi yang ada dalam KONTEKS di atas. 
    Jika informasi tidak cukup atau tidak relevan, katakan bahwa informasi tidak ditemukan.
    """
    
    # Generate jawaban
    try:
        response = ollama.chat(model=LLM_MODEL, messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt_generate}
        ])
        answer = response["message"]["content"]
    except Exception as e:
        answer = f"Maaf, terjadi kesalahan dalam menghasilkan jawaban: {str(e)}"
    
    return {
        "answer": answer,
        "sources": source_info,
        "adjusted_query": adjusted,
        "contexts_used": len(contexts)
    }

# --- Fungsi untuk memformat output dengan referensi ---
def format_response(response_data, show_detailed_sources=True):
    """Memformat response dengan jawaban dan referensi"""
    answer = response_data["answer"]
    sources = response_data["sources"]
    adjusted_query = response_data["adjusted_query"]
    contexts_used = response_data["contexts_used"]
    
    # Filter sumber dengan similarity score tinggi
    high_confidence_sources = [s for s in sources if s["similarity_score"] > 0.4]
    
    formatted_response = f"{answer}\n\n"
    formatted_response += "---\n"
    formatted_response += "**📚 Sumber Referensi:**\n\n"
    
    if high_confidence_sources:
        # Urutkan berdasarkan similarity score (descending)
        high_confidence_sources.sort(key=lambda x: x["similarity_score"], reverse=True)
        
        for i, source in enumerate(high_confidence_sources[:4]):  # Tampilkan max 4 referensi terbaik
            confidence_level = "🟢 Tinggi" if source["similarity_score"] > 0.7 else "🟡 Sedang" if source["similarity_score"] > 0.5 else "🔵 Rendah"
            
            formatted_response += f"**{i+1}. {source['source']}**\n"
            formatted_response += f"   - **Tingkat Kepercayaan:** {confidence_level}\n"
            formatted_response += f"   - **Skor Relevansi:** {source['similarity_score']:.3f}\n"
            
            if show_detailed_sources:
                formatted_response += f"   - **Cuplikan:** {source['text_preview']}\n"
            
            formatted_response += "\n"
    else:
        formatted_response += "Informasi dikumpulkan dari berbagai dokumen COVID-19 terpercaya.\n\n"
    
    # Footer informasi
    formatted_response += "---\n"
    formatted_response += f"*🔍 Berdasarkan analisis {contexts_used} dokumen | "
    formatted_response += f"Kueri: '{adjusted_query}' | "
    formatted_response += f"Tanggal: {datetime.now().strftime('%d/%m/%Y %H:%M')}*\n"
    
    return formatted_response

# --- Fungsi untuk menampilkan informasi debug ---
def show_debug_info(response_data):
    """Menampilkan informasi debug untuk developer"""
    sources = response_data["sources"]
    
    print(f"\n🔍 **INFORMASI DEBUG:**")
    print(f"📊 Jumlah dokumen yang ditemukan: {len(sources)}")
    print(f"🎯 Dokumen terbaik yang digunakan: {response_data['contexts_used']}")
    print(f"🔧 Kueri yang digunakan: '{response_data['adjusted_query']}'")
    
    print(f"\n📈 Top 5 dokumen terbaik:")
    for i, source in enumerate(sources[:5]):
        confidence_icon = "🟢" if source["similarity_score"] > 0.7 else "🟡" if source["similarity_score"] > 0.5 else "🔵"
        print(f"  {confidence_icon} {i+1}. Skor: {source['similarity_score']:.4f} | {source['source']}")
    
    # Statistik skor
    if sources:
        avg_score = sum(s["similarity_score"] for s in sources) / len(sources)
        max_score = max(s["similarity_score"] for s in sources)
        min_score = min(s["similarity_score"] for s in sources)
        print(f"\n📊 Statistik Skor: Rata-rata: {avg_score:.4f} | Maks: {max_score:.4f} | Min: {min_score:.4f}")

# --- Fungsi utama untuk chat interaktif ---
def main():
    print("=" * 70)
    print("🤖 CHATBOT COVID-19 INDONESIA")
    print("📚 Dilengkapi dengan Sistem Referensi Dokumen")
    print("=" * 70)
    print("\nSelamat datang! Silakan ajukan pertanyaan tentang COVID-19 di Indonesia.")
    print("Contoh pertanyaan:")
    print("  - 'Berapa total kasus COVID-19 di Jawa Barat?'")
    print("  - 'Apa saja gejala COVID-19 yang umum?'")
    print("  - 'Bagaimana perkembangan vaksinasi di Bali?'")
    print("  - 'Ketik 'quit' untuk keluar\n")
    
    while True:
        try:
            user_query = input("\n💬 Pertanyaan Anda: ").strip()
            
            if user_query.lower() in ['quit', 'exit', 'keluar', 'q']:
                print("Terima kasih telah menggunakan Chatbot COVID-19!")
                break
            
            if not user_query:
                print("⚠️ Silakan masukkan pertanyaan.")
                continue
            
            print("🔄 Mencari informasi...")
            
            # Jalankan RAG
            response_data = rag_chat(user_query)
            
            # Format dan tampilkan response
            formatted_answer = format_response(response_data)
            print("\n" + "=" * 70)
            print("💡 **JAWABAN:**")
            print("=" * 70)
            print(formatted_answer)
            
            # Tampilkan info debug (opsional)
            debug_choice = input("\n🔧 Tampilkan informasi detail? (y/n): ").lower()
            if debug_choice == 'y':
                show_debug_info(response_data)
            
            print("\n" + "=" * 70)
            
        except KeyboardInterrupt:
            print("\n\nTerima kasih telah menggunakan Chatbot COVID-19!")
            break
        except Exception as e:
            print(f"❌ Terjadi kesalahan: {str(e)}")
            print("Silakan coba lagi dengan pertanyaan yang berbeda.")

# --- Mode testing cepat ---
def quick_test():
    """Mode testing untuk menguji dengan pertanyaan cepat"""
    test_questions = [
        "Berapa total kasus COVID-19 di Jakarta?",
        "Apa saja gejala umum COVID-19?",
        "Bagaimana perkembangan vaksinasi di Jawa Barat?",
    ]
    
    print("🚀 MODE TESTING CEPAT")
    print("=" * 50)
    
    for i, question in enumerate(test_questions, 1):
        print(f"\n{i}. Testing: '{question}'")
        print("-" * 40)
        
        response_data = rag_chat(question)
        formatted_answer = format_response(response_data, show_detailed_sources=False)
        print(formatted_answer)
        
        # Tampilkan info singkat
        top_sources = [s for s in response_data["sources"] if s["similarity_score"] > 0.5][:2]
        if top_sources:
            print("📌 Sumber utama:")
            for source in top_sources:
                print(f"   - {source['source']} (skor: {source['similarity_score']:.3f})")
        
        if i < len(test_questions):
            input("\n⏎ Press Enter untuk test berikutnya...")

# --- EKSEKUSI PROGRAM ---
if __name__ == "__main__":
    import sys
    
    if len(sys.argv) > 1 and sys.argv[1] == "test":
        quick_test()
    else:
        main()

d:\chatbot_covid19_intern_procodecg\.venvr\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


✅ FAISS Index berhasil dimuat. Dimensi: 768, Total Vektor: 20
🤖 CHATBOT COVID-19 INDONESIA
📚 Dilengkapi dengan Sistem Referensi Dokumen

Selamat datang! Silakan ajukan pertanyaan tentang COVID-19 di Indonesia.
Contoh pertanyaan:
  - 'Berapa total kasus COVID-19 di Jawa Barat?'
  - 'Apa saja gejala COVID-19 yang umum?'
  - 'Bagaimana perkembangan vaksinasi di Bali?'
  - 'Ketik 'quit' untuk keluar

🔄 Mencari informasi...

🔧 Adjusted Query: infodemi pandemi


💡 **JAWABAN:**
 Infodemi dalam konteks pandemi COVID-19 merupakan banjir informasi palsu dan hoaks di media sosial. Saat pandemi tersebut, teori konspirasi seperti vaksin, obat herbal, dan microchip sempat menyebar luas, yang membahayakan kepercayaan masyarakat terhadap informasi yang benar. Untuk mengatasi infodemi ini, pemerintah melakukan klarifikasi fakta dan kampanye literasi digital serta menjadikan komunikasi publik lebih strategis.

---
**📚 Sumber Referensi:**

**1. Dokumen #1: 2. Program Vaksinasi Nasional Pemerintah Indones